running:  
F = open("Library_address.txt",'r')  
Library_address = F.read()   
%run $Library_address/tmm_fitting.ipynb

In [1]:
F = open("Library_address.txt",'r') 
Library_address = F.read()
%run $Library_address/data_io.ipynb
%run $Library_address/notifier.ipynb
import time
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import sys
sys.path.insert(0, 'C:/Python27/tmm/')
import tmm_core
from scipy.optimize import curve_fit

In [2]:
def norm_gaussian(xeV,xeV0,w):
    # Gaussian with integral 1
    
    sigma = 1/2.355 * w * 1e-3
    gauss = 1/(np.sqrt(2*np.pi)*sigma) * np.exp(-1/2*((xeV-xeV0)/sigma)**2)
    
    return gauss

def lorentzian_for_eps(xeV,xeV0,w,f):
    
    denominator=(xeV0**2-xeV**2)**2+(xeV**2*w**2)
    conw =0
    if conw != 0:
        gauss = norm_gaussian(xeV,xeV0,conw)
        denominator = np.convolve(denominator, gauss, 'same')
    
    real=f*(xeV0**2-xeV**2)/denominator
    imag=xeV*w*f/denominator
    
    return real+1j*imag

def eps_TMD(energy, *pars):
    
    n_peaks = (len(pars)-1)//3 # background dielectric constant + 3 parameters per peak
    
    bck=pars[0]
    centers=pars[1:n_peaks+1]
    widths=pars[n_peaks+1:2*n_peaks+1]
    osc=pars[2*n_peaks+1:3*n_peaks+1]
    
    epsilon=bck
  
    for i in range(len(centers)):
        epsilon += lorentzian_for_eps(energy, centers[i], widths[i], osc[i])
    return epsilon

def calculate_tmm (energy, n_peaks, *pars, n=[], thickness_list=[inf,inf], calculate='R'):
    #    energy is the independent variable of fitting - the energy axis in the measurement
    #    centersi - center energies of the 3 lorentzians that superimpose the dielectric function of the TMD
    #    widthsi -  widths of those lorentzians
    #    osci - the oscillator sterngths of these lorenzians
    #    bck - background constant dielectric value for the TMD
    
    th_eff = 0
    pol ='s'
    #print(pars)
    if pars != ():
        bck=pars[0]
        centers=pars[1:n_peaks+1]
        widths=pars[n_peaks+1:2*n_peaks+1]
        osc=pars[2*n_peaks+1:3*n_peaks+1]
        

        #centers = pars['centers']
        #widths = pars['widths']
        #osc = pars['amplitudes']
        #bck = pars['bck']
        thickness_list=[inf,t_sample, inf]
        n1 = np.sqrt(eps_TMD (energy, *pars))

    
    quantity = np.zeros(len(energy))
    
    for e in range(len(energy)):
        
        if pars != () and n != []: # material's substrate is not air
            n_list = np.array([1, n1[e], n[e]] )
        elif pars != (): # material's substrate is air
            n_list = np.array([1, n1[e], 1] )
        elif n != [] : # substrate calculation only
            n_list = np.array([1, n[e]])
#       
        #print(n_list, thickness_list)
        quantity[e] = tmm_core.coh_tmm(pol, n_list, thickness_list, th_eff*2*np.pi/360, 1240/energy[e])[calculate]
                 
    return quantity

def is_pars_reasonable(pars):
    if len(pars['labels']) != len(pars['centers'][0]) or len(pars['centers'][0]) != len(pars['widths'][0]) or len(pars['widths'][0]) != len(pars['amplitudes']):
        raise ValueError('Inconsistent number of parameters for the peaks!')
    if not np.all(np.diff(pars['centers']) > 0):
        print(pars['centers'])
        raise ValueError('Peaks are not monotonically ordered')
    if any(pars['amplitudes'] < osc_low_bounds):
        raise ValueError('Too large lower bounds for amplitudes : some peaks may be fit with negative amplitudes')
        
    return

def print_fit_report(*p0_fit, file_name='data'): # Print and Export the fit results
    
    #print(Zero)
    # Prepare the rows of the report as the peak values
    report_index = ['Substrate', 'Date', 'Objective', 'Comment','Layer', 'Flake', 'Diameter', 'Pressure']
    report_index += ['Background'] + [ peak_label +'_' + value for value in ['center', 'width', 'amp'] for peak_label in pars['labels'] ]
    #other_index = ['A_1s_2s_center','A_1s_3s_center','A_1s_4s_center', 'A_B_center', 'A_B_2s_center', 'A_C_center', 'B_1s_2s_center' ]
    
    # labels for difference between A_1s & other peak centers
    other_index = ['A_1s_'+ peak_label+'_center' for peak_label in pars['labels'] if peak_label != 'A_1s']
    
    # labels for difference between B_1s & other peak centers indexed after B_1s in pars['labels']
    i_B_1s = 0
    if 'B_1s' in pars['labels']:
        i_B_1s = pars['labels'].index('B_1s')
        other_index += ['B_1s_'+ peak_label+'_center' for peak_label in pars['labels'] if pars['labels'].index(peak_label) > i_B_1s]
    
    # labels for relative amplitude of A_1s & other peaks
    other_index += [peak_label+'/A_1s_amp' for peak_label in pars['labels'] if peak_label != 'A_1s']
    

    report_index += other_index
    
    fit_report = pd.DataFrame(index=report_index)
    
    
    for i in range(n2-n1+1):
        p=p0_fit[i]
        # difference between A_1s & other peak centers
        other_report = tuple( p[ pars['labels'].index(peak_label) + 1 ] - p[1] for peak_label in pars['labels'] if peak_label != 'A_1s' )
        
        # difference between B_1s & other peak centers indexed after B_1s in pars['labels']
        if i_B_1s:
            other_report += tuple( p[ pars['labels'].index(peak_label) + 1 ] - p[i_B_1s+1] for peak_label in pars['labels'] if pars['labels'].index(peak_label) > i_B_1s )
        
        # labels for relative amplitude of A_1s & other peaks
        other_report += tuple( p[ pars['labels'].index(peak_label) + 2*n_peaks+1] / p[2*n_peaks+1] for peak_label in pars['labels'] if peak_label != 'A_1s')
        
        fit_report[text_for_originpro(data_labels[n1+i])] = np.hstack(( 0,0,0,0,0,0,0,0, p, other_report)) # 0s are temporary for the following 4 parameters ['Layer', 'Flake', 'Diameter', 'Pressure']
        
    fit_report.loc['Substrate'] = [substrate]*(n2-n1+1) if n2-n1+1 > 1 else substrate
    fit_report.loc['Date'] = [date]*(n2-n1+1) if n2-n1+1 > 1 else date
    fit_report.loc['Objective'] = [objective]*(n2-n1+1) if n2-n1+1 > 1 else objective
    fit_report.loc['Comment'] = [comment]*(n2-n1+1) if n2-n1+1 > 1 else comment
    fit_report.loc['Layer'] = layers[n1:n2+1]
    fit_report.loc['Flake'] = flakes[n1:n2+1]
    fit_report.loc['Diameter'] = diameters[n1:n2+1]
    fit_report.loc['Pressure'] = pressures[n1:n2+1]
    
    fig = plt.figure()
    plt.plot(fit_report.loc['A_1s_center'], 1000*fit_report.loc['A_1s_width'], label='A exciton', marker='o', markersize=12, color="red")
    plt.xlabel('Peak Energy (eV)')
    plt.ylabel('Linewidth (meV)')
    fig.set_size_inches(w=8,h=6)
    plt.show()
    
    fit_report.insert(loc=0, column='Units', value=['']*4 + ['Monolayer', '#', '\g(m)m', 'psi'] +['']+ ['eV']*2*n_peaks + ['']*(len(fit_report.index)-2*n_peaks-9)) 
    fit_report.columns.name = 'Sample'
    export_pd(fit_report.transpose(), file_name='fit_report_' + file_name, index=True, header=True)
    
    return fit_report

def plot_fits(n, start_time, energy, n_peaks, R_sample, *p0_fit, n_fit = []):
    
    bck_fit = p0_fit[0]
    centers_fit = p0_fit[1:n_peaks+1]
    widths_fit = [w*1000 for w in p0_fit[n_peaks+1:2*n_peaks+1]]
    osc_fit = p0_fit[2*n_peaks+1:3*n_peaks+1]
    
    x_max = max(energy)
    x_min = min(energy)
    
    end_time = time.time()
    text_fit_results  = 'Fit #%s of %s took %.2f seconds\n' %(n+1, N, end_time - start_time)
    text_fit_results += 'Background Dielectric: %s\n' %np.round(bck_fit ,2)
    text_fit_results += 'Energies (eV): %s\n' %np.round(centers_fit ,4)
    text_fit_results += 'Widths (meV): %s\n' %np.round(widths_fit,2)
    text_fit_results += 'Amplitudes: %s' %np.round(osc_fit,2)
    
    R_sample_d = np.diff(R_sample,axis=0)
    energy_d = energy[1:]
    
    fit_RC = calculate_tmm(energy, n_peaks, *p0_fit, n=n_fit, calculate='R')
    fit_RC_d = np.diff(fit_RC)
    fit_Abs = calculate_tmm(energy, n_peaks, *p0_fit, calculate='1-R-T')# n=n_fit is not used -> Absorption of free-standing
    epsilon_fit = eps_TMD(energy, n_peaks, *p0_fit)
    
    p0_fit_no_A1s = np.copy(p0_fit); p0_fit_no_A2s = np.copy(p0_fit); p0_fit_no_A3s = np.copy(p0_fit); p0_fit_no_A2s_3s = np.copy(p0_fit); p0_fit_no_A2s_3s = np.copy(p0_fit)
    p0_fit_no_A1s[2*n_peaks+1] =0 #0 for oscillator strength of A_1s
    p0_fit_no_A2s[2*n_peaks+2] =0 #0 for oscillator strength of A_2s
    p0_fit_no_A3s[2*n_peaks+3] =0 #0 for oscillator strength of A_3s
    p0_fit_no_A2s_3s[2*n_peaks+2:2*n_peaks+4] = 0 #0 for oscillator strength of A_2s, A_3s
    
    fit_RC_no_A1s = calculate_tmm(energy, n_peaks, *p0_fit_no_A1s, n=n_fit, calculate='R')
    fit_RC_no_A2s = calculate_tmm(energy, n_peaks, *p0_fit_no_A2s, n=n_fit, calculate='R')
    fit_RC_no_A3s = calculate_tmm(energy, n_peaks, *p0_fit_no_A3s, n=n_fit, calculate='R')
    fit_RC_no_A2s_3s = calculate_tmm(energy, n_peaks, *p0_fit_no_A2s_3s, n=n_fit, calculate='R')

     
    fig = plt.figure(1)
    # set up subplot grid
    grid = plt.GridSpec(6,3,  wspace=0.4, hspace=0.6)

    plt.subplot(grid[0:3, 0:2])
    for x_line in centers_fit: # vertical lines at the fit peak centers
        if x_line < x_max: # only show the peaks within the range of fit 
            plt.axvline(x=x_line, color='r', ls='--', lw=0.5)
            
    plt.plot(energy, R_sample, label='Data')
    plt.plot(energy, fit_RC, label='Fit')
    plt.plot(energy, fit_RC_no_A1s, label='A$_{1s}$=0', ls='--')
    plt.plot(energy, fit_RC_no_A2s, label='A$_{2s}$=0')
    plt.plot(energy, fit_RC_no_A3s, label='A$_{3s}$=0')
    plt.plot(energy, fit_RC_no_A2s_3s, label='A$_{2s}$, A$_{3s}$=0', ls='--')
    
    fit_pd = pd.DataFrame( np.transpose([energy, R_sample, fit_RC, fit_RC_no_A1s, fit_RC_no_A2s, fit_RC_no_A3s, fit_RC_no_A2s_3s ]), columns=['Energy (eV)', 'Data', 'Fit', 'A_1s=0', 'A_2s=0', 'A_3s=0', "A_2s=0, A_3s=0"] )
    export_pd(fit_pd, file_name='fit_result_' + '%s Data & Fit' %(n1+n), index=False, header=True)

    plt.legend()
    plt.title('%s Data & Fit' %data_labels[n1+n])
    plt.xlabel('Energy (eV)')
    plt.text( x_max*0.4+x_min*0.6, max(R_sample)*0.8+min(R_sample)*0.2, text_fit_results)
    
    plt.subplot(grid[3:6, 0:2])
    for x_line in centers_fit: # vertical lines at the fit peak centers
        if x_line < x_max: # only show the peaks within the range of fit 
            plt.axvline(x=x_line, color='r', ls='--', lw=0.5)
    plt.plot(energy_d, R_sample_d)
    plt.plot(energy_d, fit_RC_d)   
    plt.title('Derivatives of Measurement and fit')
    plt.xlabel('Energy (eV)')
    

    plt.subplot(grid[0:2, 2])
    plt.plot(energy, bck_fit+epsilon_fit.real, label='Real')
    plt.plot(energy, epsilon_fit.imag, label='Imaginary')
    plt.plot([x_min, x_max], [bck_fit, bck_fit], label='Background')
    plt.legend()
    plt.title('Dielectric Function (Fit)')
    plt.xlabel('Energy (eV)')

    plt.subplot(grid[2:4, 2])
    plt.plot([x_min, x_max], [0,0])
    plt.plot(energy, 100*fit_Abs)
    plt.title('Absorption of Suspended 1L (Fit)')
    plt.xlabel('Energy (eV)')
    plt.ylabel('Absorption (%)')
    plt.tight_layout()
    
    plt.subplot(grid[4:6, 2])
    for x_line in centers_fit: # vertical lines at the fit peak centers
        if x_line < x_max: # only show the peaks within the range of fit 
            plt.axvline(x=x_line, color='r', ls='--', lw=0.5)
    plt.plot(energy, 100*R_sample/fit_RC-100)
    plt.plot([x_min, x_max], [0,0])
    plt.title('Residual of Data-Fit')
    plt.ylabel('Data/Fit - 1 (%)')
    plt.xlabel('Energy (eV)')
    
    #fig.tight_layout()
    fig.set_size_inches(w=16,h=12)
    plt.show()
       

NameError: name 'inf' is not defined